In [2]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 637, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 637 (delta 136), reused 184 (delta 109), pack-reused 363
Receiving objects: 100% (637/637), 10.76 MiB | 6.28 MiB/s, done.
Resolving deltas: 100% (321/321), done.


# 작업형 제 1유형

## 다음은 California Housing 데이터 세트이다. 데이터 중 결측치가 있는 경우 해당 데이터의 행을 모두 제거하고, 첫 번째 행부터 순서대로 70%까지의 데이터를 훈련 데이터로 추출한 데이터 세트를 구성한다. 변수 중 housing_median_age의 Q1 값을 정수로 계산하시오

In [1]:
import pandas as pd
df = pd.read_csv('yemoonsaBigdata/datasets/Part3/301_housing.csv')

# 결측치 제거
df = df.dropna()
df = df.reset_index()
# 70%까지
ind = int(len(df)*(70 / 100))
train = df.iloc[:ind]
res = int(train['housing_median_age'].quantile(0.25))
print(res)

19


## 다음은 국가별 인도별 인구 10만 명당 결핵 유병률 데이터 세트이다. 2000년도의 국가별 결행 유병률 데이터 세트에서 2000년도의 평균값보다 더 큰 유병률 값을 가진 국가의 수를 계산하시오

In [ ]:
import pandas as pd
df = pd.read_csv('yemoonsaBigdata/datasets/Part3/302_worlddata.csv')
res_df = df[df['year'] == 2000].T.drop('year')
res = len(res_df[res_df > res_df.mean()].dropna())
print(res)

76


## 다음은 Titanic 데이터 세트이다. 주어진 데이터 세트의 컬럼 중 빈 값 또는 결측치를 확인하여 결측치의 비율이 가장 높은 변수명을 출력하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('yemoonsaBigdata/datasets/Part3/303_titanic.csv')
res = df.isnull().sum().sort_values(ascending = False).index[0]
print(res)

Age


# 작업형 제 2유형
다음은 travel Insurance 데이터 세트이다. 주어진 훈련 데이터 세트를 이용하여 고객별 여행보험 가입 여부 예측 모형을 만들고, 가장 높은 Accuracy 값을 가지는 최종 모델을 도출하시오. 해당 모델을 활용하여 보험 가입 여부 예측 값을 계산하고 결괏값은 csv 파일로 제출(roc_auc_score)

## 방법 1. onehotencoder

In [ ]:
import pandas as pd
x_test = pd.read_csv('yemoonsaBigdata/datasets/Part3/304_x_test.csv')
x_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/304_x_train.csv')
y_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/304_y_train.csv')

In [ ]:
# 결측치 X, 이상치 skip
# 명목형
from sklearn.preprocessing import OneHotEncoder

col_drop = ['ID']
col_num = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
col_cat = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']

x_train = x_train.drop(col_drop, axis = 1)
x_test = x_test.drop(col_drop, axis = 1)
y_train = y_train.drop(col_drop, axis = 1)

x = pd.concat([x_train, x_test])
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(x[col_cat])

x_train_res = ohe.transform(x_train[col_cat])
x_test_res = ohe.transform(x_test[col_cat])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())

x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

In [ ]:
# split, scaler, model
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 123, n_estimators = 25)
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred = model.predict(x_val)
print(accuracy_score(y_pred, y_val))

# y_tr_pred = model.predict(x_tr)
# print(accuracy_score(y_tr, y_tr_pred))

from sklearn.metrics import roc_auc_score
y_pred = model.predict_proba(x_val)[:, 1]
print(roc_auc_score(y_val, y_pred))

y_pred_train = model.predict_proba(x_tr)[:, 1]
print(roc_auc_score(y_tr, y_pred_train))

0.7829977628635347
0.7842430149447693
0.9883040935672514


In [ ]:
y_pred = model.predict_proba(x_test_fin)[:, 1]
result = pd.DataFrame({'index' : x_test_fin.index, 'y_pred' : y_pred})
result.to_csv('00300.csv', index = False)

In [ ]:
result

,index,y_pred
0,0,0.230
1,1,0.280
2,2,0.230
3,3,0.230
4,4,0.515
...,...,...
492,492,0.310
493,493,0.230
494,494,0.320
495,495,0.120


## 방법  2. labelencoder 

In [3]:
import pandas as pd
x_test = pd.read_csv('yemoonsaBigdata/datasets/Part3/304_x_test.csv')
x_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/304_x_train.csv')
y_train = pd.read_csv('yemoonsaBigdata/datasets/Part3/304_y_train.csv')

# 결측치 X, 이상치 skip
col_drop = ['ID']
col_num = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
col_cat = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']

x_train = x_train.drop(col_drop, axis = 1)
x_test = x_test.drop(col_drop, axis = 1)
y_train = y_train.drop(col_drop, axis = 1)

# split, scaler, model
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test[col_num] = scaler.transform(x_test[col_num])

from sklearn.preprocessing import LabelEncoder
x = pd.concat([x_train[col_cat], x_test[col_cat]])

for col in col_cat:
    le = LabelEncoder()
    le.fit(x[col])
    x_tr[col] = le.transform(x_tr[col])
    x_val[col] = le.transform(x_val[col])
    x_test[col] = le.transform(x_test[col])

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state = 123, n_estimators = 25)
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred = model.predict(x_val)
print(accuracy_score(y_pred, y_val))

# y_tr_pred = model.predict(x_tr)
# print(accuracy_score(y_tr, y_tr_pred))

from sklearn.metrics import roc_auc_score
y_pred = model.predict_proba(x_val)[:, 1]
print(roc_auc_score(y_val, y_pred))

y_pred_train = model.predict_proba(x_tr)[:, 1]
print(roc_auc_score(y_tr, y_pred_train))

0.7762863534675615
0.7854559237600174
0.9889028125870231


In [ ]:
y_pred = model.predict_proba(x_test)[:, 1]
result = pd.DataFrame({'index' : x_test.index, 'y_pred' : y_pred})
result.to_csv('00300.csv', index = False)